
$\qquad$ $\qquad$$\qquad$  **TDA206/DIT206 Discrete Optimization: Home Assignment 2 -- Integer LP and Relaxation** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Marc Constantin** <br />
$\qquad$ $\qquad$$\qquad$                   **Due Date: 17th Feb** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Name, Personal No., Email** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.
(5 points) 
There are 4 space colonies, each of which  requires a certain number of plasma conduits. There are 3 starbases in the vicinity. Each of them has total number of conduits they can spare and supply to the colonies. For each pair of starbase and colony, there is an associated cost for sending a cargo ship  (each of which carries one plasma conduit), as shown in the table below:


\begin{array}{l|c|c|c|c|c} 
      & Triacus & New Berlin  & Strnad  & Vega  & supply\\ \hline
 Farpoint &   6 &  9 & 10 & 8 & 35\\
 Yorktown &  9 & 5 & 16 & 14 & 40\\
 Earhart & 12 &  7 & 13 & 9 & 50\\ \hline
    demand & 20 &30&30&45& \left(\sum=125\right) \\ 
\end{array}

Your goal is to supply the colonies the plasma conduits they need, at minimum cost.


Formulate a LP to solve the problem, explain why the solution is integral with a proof.

### LP formulation (transportation problem)

Let the set of starbases be  
$$
I=\{\text{Farpoint},\text{Yorktown},\text{Earhart}\}
$$
and the set of colonies be  
$$
J=\{\text{Triacus},\text{NewBerlin},\text{Strnad},\text{Vega}\}.
$$

For each pair \((i,j)\in I\times J\), let \(c_{ij}\) be the shipping cost per conduit (as given in the table).  
Let \(s_i\) be the supply at starbase \(i\) and \(d_j\) the demand at colony \(j\).

Decision variables  
$$
x_{ij}\ge 0 \qquad \text{for all } (i,j)\in I\times J,
$$
where \(x_{ij}\) denotes the number of plasma conduits shipped from starbase \(i\) to colony \(j\).

We minimize total shipping cost:
$$
\min \sum_{i\in I}\sum_{j\in J} c_{ij}\,x_{ij}.
$$

Supply constraints (all available conduits are shipped since \(\sum_i s_i=\sum_j d_j\)):
$$
\sum_{j\in J} x_{ij} = s_i \qquad \text{for all } i\in I.
$$

Demand constraints(each colony receives exactly its demand):
$$
\sum_{i\in I} x_{ij} = d_j \qquad \text{for all } j\in J.
$$


### Why an optimal solution is integral

This LP is a minimum-cost flow (transportation) problem on a bipartite network:
nodes on the left are starbases, nodes on the right are colonies, and each arc \((i,j)\)
carries flow \(x_{ij}\).

Let \(A\) be the node–arc incidence matrix for the equalities above. Each column corresponds
to one variable \(x_{ij}\) and has exactly two nonzero entries: one \(+1\) in the row for starbase \(i\)
and one \(+1\) in the row for colony \(j\) (using a consistent sign convention).

From the theory of totally unimodular matrices / network flows: the incidence matrix of a bipartite graph
is totally unimodular (TU). If \(A\) is TU and the right-hand side \(b\) is integral, then every basic feasible
solution of \(\{x\ge 0: Ax=b\}\) is integral. Since an LP attains an optimum at a basic feasible solution whenever
an optimum exists, there is an optimal solution with
$$
x_{ij}\in\mathbb{Z}\quad \text{for all } (i,j).
$$

Equivalently, by the integrality theorem for min-cost flows: with integral supplies/demands (and capacities, if present),
an optimal min-cost flow exists that is integral.






# Question 2.

Recall the Minimum Weight Vertex Cover (VC) Problem: Given an undirected graph $G=(V, E)$, with node set $V$ and edge set $E$, where each node has a positive weight $w(v)$ associated with it (see the attached figure), the goal is to select a subset $V'\subseteq V$ of nodes such that every edge has at least one node incident to it, and the total selected node weight $\sum_{v\in V'} w(v)$ is minimized. 

* (4 points) Formulate the ILP for the VC problem for the attached graph, and solve it using **CVXPY** integer solver, for instance, `myVar = cp.Variable(<dim>, integer=True)`.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY** and comment on the relation between the two solutions.
* (2 points) Apply the rounding rule discussed in class to the optimal LP solution to obtain a solution to the ILP and compare it to the optimal ILP solution.

<!-- ![title](vertex_cover_example.png) -->


In [ ]:
import cvxpy as cp
import numpy as np


V = [1, 2, 3, 4, 5, 6]
w = {1:1, 2:3, 3:4, 4:2, 5:2, 6:4}
E = [(1,2),(2,3),(2,5),(3,5),(3,4),(4,5),(5,6)]
idx = {v:i for i,v in enumerate(V)}
n = len(V)

print("Installed solvers:", cp.installed_solvers())

# (A) ILP

x_ilp = cp.Variable(n, boolean=True)

constraints_ilp = [x_ilp[idx[u]] + x_ilp[idx[v]] >= 1 for (u,v) in E]
objective_ilp = cp.Minimize(sum(w[v]*x_ilp[idx[v]] for v in V))

prob_ilp = cp.Problem(objective_ilp, constraints_ilp)
prob_ilp.solve(solver=cp.GLPK_MI)

ilp_cover = [v for v in V if x_ilp.value[idx[v]] > 0.5]
print("\n=== ILP ===")
print("MIP solver used: GLPK_MI")
print("ILP optimum:", prob_ilp.value)
print("ILP cover:", ilp_cover)

# (B) LP relaxation

x_lp = cp.Variable(n)

constraints_lp = [x_lp >= 0, x_lp <= 1]
constraints_lp += [x_lp[idx[u]] + x_lp[idx[v]] >= 1 for (u,v) in E]

objective_lp = cp.Minimize(sum(w[v]*x_lp[idx[v]] for v in V))
prob_lp = cp.Problem(objective_lp, constraints_lp)
prob_lp.solve(solver=cp.ECOS)

print("\n=== LP relaxation ===")
print("LP solver used: ECOS")
print("LP optimum:", prob_lp.value)
print("LP x*:", np.round(x_lp.value, 6))

# (C) Rounding rule
x_star = x_lp.value
rounded_cover = [v for v in V if x_star[idx[v]] >= 0.5]
rounded_cost = sum(w[v] for v in rounded_cover)

print("\n=== Rounding (x >= 1/2) ===")
print("Rounded cover:", rounded_cover)
print("Rounded cost:", rounded_cost)


Installed solvers: ['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

=== ILP ===
MIP solver used: GLPK_MI
ILP optimum: 7.0
ILP cover: [1, 3, 5]

=== LP relaxation ===
LP solver used: ECOS
LP optimum: 6.999999999676878
LP x*: [ 0.501446  0.498554  0.501446  0.498554  1.       -0.      ]

=== Rounding (x >= 1/2) ===
Rounded cover: [1, 3, 5]
Rounded cost: 7


## Question 2 – Minimum Weight Vertex Cover

We are given an undirected graph G = (V, E) where each vertex v has a positive weight w(v).  
The goal is to choose a set of vertices (a vertex cover) so that every edge has at least one endpoint chosen, and the total weight is minimized.

Vertices: V = (1, 2, 3, 4, 5, 6)  
Weights: w(1)=1, w(2)=3, w(3)=4, w(4)=2, w(5)=2, w(6)=4  
Edges:
$$
E = \{(1,2),(2,3),(2,5),(3,5),(3,4),(4,5),(5,6)\}.
$$



### (a) ILP formulation and optimal solution

I use one decision variable per vertex:
- x_v = 1 if vertex v is selected
- x_v = 0 otherwise

The ILP is:
$$
\min \sum_{v \in V} w(v)\,x_v
$$
subject to the vertex cover constraints (one per edge):
$$
x_u + x_v \ge 1 \quad \text{for every edge } (u,v)\in E,
$$
and integrality:
$$
x_v \in \{0,1\} \quad \text{for all } v \in V.
$$

Result (CVXPY with GLPK_MI):
- Optimal ILP value: 7.0
- One optimal vertex cover: vertices 1, 3, and 5



### (b) LP relaxation and relation to the ILP

The LP relaxation replaces x_v in {0,1} with:
$$
0 \le x_v \le 1.
$$

So the LP is the same objective and edge constraints, but with 0 <= x_v <= 1.

Result (CVXPY with ECOS):
- LP optimal value: about 7.0 (numerically 6.999999999676878)
- LP solution (approximately):
$$
x^* \approx (0.501446,\ 0.498554,\ 0.501446,\ 0.498554,\ 1,\ 0).
$$

Comment:
The LP relaxation gives a lower bound on the ILP optimum. In this instance the LP value is essentially the same as the ILP value (both are about 7), so the relaxation is tight here. The small differences from exact halves come from numerical solver tolerance.



### (c) Rounding rule and comparison

Using the standard rounding rule from class for vertex cover:
select every vertex with x_v >= 0.5.

From the LP solution this gives:
- Rounded cover: vertices 1, 3, and 5
- Rounded cost: 1 + 4 + 2 = 7

Comparison:
The rounded solution matches the optimal ILP solution (same chosen vertices and same total cost).


# Question 3.

Consider a number of interpreters (Olof, Petra, Qamar,
  Rachel, Soren and Tao), as well as a set of languages (Arab,
  Bengali, Cantonese, Dutch, English, French and German). Each
  interpreter speaks a number of different languages (abbreviated by
  first letter), and has a certain per-diem integer cost:

\begin{array}{lll}
Interpreter & Languages & Cost\\
O & ABD & 3\\
P & C & 1\\
Q & CDG & 1\\
R & B & 2\\
S & G & 4\\
T & EF & 1\\
\end{array}

* (2 points) A *hypergraph* is a structure $H = (V,E)$ where $V$ is a set of vertices and $E$ is a collection of subsets of $V$. The special case when all subsets $e \in E$ have size exactly $2$ corresponds to the familiar case of a graph. A vertex cover in such a hypergraph is a subset $U \subseteq V$ such that $e \cap U \not = \emptyset$ for each $e \in E$ (note that this reduces to the usual vertex cover in graphs). Show that the problem of finding interpreters can be formulated as a vertex cover problem in a sutable hypergraph.
* (4 points) Develop a ILP formulation to finding the vertex cover of minimum cost in a hypergraph. The hypergraph can be represented as a $|V| \times |E|$ binary matrix $A$ where $A[i,j] = 1$ iff vertex $i$ is in edge $j$ and 0 otherwise. The costs for vertices are in an array $\texttt{c}$ where the cost of picking vertex $i$ is $c[i]$. Use the ILP formulation for the VC problem to hire the cheapest set of interpreters such that all languages are covered. Input the data above manuallly and solve it using **CVXPY**'s integer solver.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY**.
* (2 points) Explain why the two solutions above are same (different).


In [12]:
import cvxpy as cp
import numpy as np

V = ["O", "P", "Q", "R", "S", "T"]

L = ["A", "B", "C", "D", "E", "F", "G"]

c = np.array([3, 1, 1, 2, 4, 1], dtype=float)  # O,P,Q,R,S,T

A = np.array([
    # A  B  C  D  E  F  G
    [ 1, 1, 0, 1, 0, 0, 0],  
    [ 0, 0, 1, 0, 0, 0, 0],  
    [ 0, 0, 1, 1, 0, 0, 1],  
    [ 0, 1, 0, 0, 0, 0, 0],  
    [ 0, 0, 0, 0, 0, 0, 1],  
    [ 0, 0, 0, 0, 1, 1, 0],  
], dtype=float)

print("Installed solvers:", cp.installed_solvers())

# (b) ILP: minimum-cost vertex cover in hypergraph
x_ilp = cp.Variable(len(V), boolean=True)

constraints_ilp = [A.T @ x_ilp >= 1]  
obj_ilp = cp.Minimize(c @ x_ilp)

prob_ilp = cp.Problem(obj_ilp, constraints_ilp)
prob_ilp.solve(solver=cp.GLPK_MI)

x_ilp_val = (x_ilp.value > 0.5).astype(int)
chosen_ilp = [V[i] for i in range(len(V)) if x_ilp_val[i] == 1]

print("\n=== ILP (GLPK_MI) ===")
print("Optimal cost:", prob_ilp.value)
print("Chosen interpreters:", chosen_ilp)
print("x:", x_ilp_val)

# (c) LP relaxation
x_lp = cp.Variable(len(V))

constraints_lp = [
    A.T @ x_lp >= 1,
    x_lp >= 0,
    x_lp <= 1
]
obj_lp = cp.Minimize(c @ x_lp)

prob_lp = cp.Problem(obj_lp, constraints_lp)
prob_lp.solve(solver=cp.ECOS)

print("\n=== LP relaxation (ECOS) ===")
print("Optimal cost:", prob_lp.value)
print("x*:", np.round(x_lp.value, 6))

chosen_lp = [V[i] for i in range(len(V)) if x_lp.value[i] >= 0.5]
print("Interpreters with x >= 0.5:", chosen_lp)


Installed solvers: ['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

=== ILP (GLPK_MI) ===
Optimal cost: 5.0
Chosen interpreters: ['O', 'Q', 'T']
x: [1 0 1 0 0 1]

=== LP relaxation (ECOS) ===
Optimal cost: 5.000000000139226
x*: [ 1.  0.  1.  0. -0.  1.]
Interpreters with x >= 0.5: ['O', 'Q', 'T']


## Question 3 — Interpreter hiring

We have interpreters \(O,P,Q,R,S,T\) and languages \(A,B,C,D,E,F,G\).  
Each interpreter speaks a set of languages and has a cost. The goal is to hire a minimum-cost set of interpreters such that every language is covered.



### (a) Hypergraph vertex cover model

Define a hypergraph \(H=(V,E)\) where:

- Vertices \(V\) are the interpreters:
  \[
  V=\{O,P,Q,R,S,T\}.
  \]

- For each language \(\ell \in \{A,B,C,D,E,F,G\}\), create one hyperedge \(e_\ell\) containing all interpreters that speak \(\ell\).
  Concretely:
  \[
  e_A=\{O\},\quad
  e_B=\{O,R\},\quad
  e_C=\{P,Q\},\quad
  e_D=\{O,Q\},\quad
  e_E=\{T\},\quad
  e_F=\{T\},\quad
  e_G=\{Q,S\}.
  \]

Hiring a set of interpreters \(U\subseteq V\) that covers all languages is exactly a vertex cover in this hypergraph:
\[
U \cap e_\ell \neq \emptyset \quad \text{for every language } \ell.
\]
We want a minimum-cost vertex cover (costs on vertices).



### (b) ILP formulation and optimal solution

Let \(x_i \in \{0,1\}\) indicate whether interpreter \(i\) is hired.

Min-cost hypergraph vertex cover:
\[
\min \sum_{i\in V} c_i x_i
\]
subject to one constraint per language \(\ell\):
\[
\sum_{i\in e_\ell} x_i \ge 1 \quad \text{for all } \ell \in \{A,B,C,D,E,F,G\},
\]
and
\[
x_i \in \{0,1\} \quad \text{for all } i\in V.
\]

**Result (CVXPY + GLPK\_MI):**
- Optimal cost: \(5.0\)
- Chosen interpreters: \(O, Q, T\)



### (c) LP relaxation and result

Relax integrality to:
\[
0 \le x_i \le 1 \quad \text{for all } i\in V.
\]

**Result (CVXPY + ECOS):**
- LP optimal cost: \(5.000000000139226 \approx 5\)
- LP solution is integral: \((1,0,1,0,0,1)\), so the chosen interpreters are again \(O,Q,T\).



### (d) Why ILP and LP give the same answer here

Some languages force specific choices:

- Language \(A\) is spoken only by \(O\), so \(O\) must be hired.
- Languages \(E\) and \(F\) are spoken only by \(T\), so \(T\) must be hired.

After hiring \(O\) and \(T\), the remaining uncovered language is \(G\), which can be covered by either \(Q\) or \(S\).  
Since \(c(Q)=1\) and \(c(S)=4\), choosing \(Q\) is cheapest.

Therefore the unique minimum-cost solution is to hire \(O, Q, T\) with total cost \(3+1+1=5\), and the LP relaxation ends up with the same integral optimum.


# Question 4. 
Consider the ILP and its LP relaxation corresponding to the VC problem for the graph $G$ given in the data file. This is a ***random graph*** $G(n,p)$ with $n=100$ vertices generated as follows: for each pair of vertices **independently**, we add an edge with probability $p=0.1$ (so the graph has about 1000 edges).

* **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.
* **b**. (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).
* **c**. (6 points) Consider the following rounding rule: we build up the vertex cover incrementally starting with $S:= \emptyset$. Now consider the edges in $G$ in any order. If an edge $(u,v)$ is already covered by a vertex in $S$, do nothing. Otherwise add to $S$ the vertex $u$ if $x^*(u) \geq x^*(v)$, or $v$ otherwise (where ${\bf x}^*$ is the LP optimum solution computed in part (b).  Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b). Compare the cost of the solution produced by this rule to the optimal solution.

In [ ]:
import numpy as np
import cvxpy as cp
import re

data_path = "graph.txt"
tol = 1e-8  

def load_graph_flexible(path):
    lines = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            nums = re.findall(r"-?\d+", line)
            if nums:
                lines.append([int(x) for x in nums])
    if not lines:
        raise ValueError("No numeric data found in file.")

    n0 = len(lines)
    if all(len(row) == n0 for row in lines):
        A = np.array(lines, dtype=int)
        E = list(map(tuple, np.transpose(np.triu(A, 1).nonzero())))
        return E, A.shape[0]

    edges = np.array([row[:2] for row in lines if len(row) >= 2], dtype=int)

    if edges.min() == 1:
        edges -= 1

    
    edges = edges[edges[:, 0] != edges[:, 1]]
    u = np.minimum(edges[:, 0], edges[:, 1])
    v = np.maximum(edges[:, 0], edges[:, 1])
    edges = np.unique(np.stack([u, v], axis=1), axis=0)

    n = int(edges.max() + 1)
    return [tuple(e) for e in edges], n

def is_vertex_cover(S, E):
    S = set(S)
    return all((u in S) or (v in S) for (u, v) in E)

E, n = load_graph_flexible(data_path)
print(f"Loaded graph with n={n}, |E|={len(E)}")
print("Installed solvers:", cp.installed_solvers())

# (a) ILP
x_ilp = cp.Variable(n, boolean=True)
constraints_ilp = [x_ilp[u] + x_ilp[v] >= 1 for (u, v) in E]
prob_ilp = cp.Problem(cp.Minimize(cp.sum(x_ilp)), constraints_ilp)
prob_ilp.solve(solver=cp.GLPK_MI)

x_ilp_val = (x_ilp.value > 0.5).astype(int)
S_ilp = np.where(x_ilp_val == 1)[0]
cost_ilp = len(S_ilp)

print("\n=== (a) ILP ===")
print("ILP optimum (size):", cost_ilp)
print("ILP is cover?", is_vertex_cover(S_ilp, E))

# (b) LP relaxation (use GLPK for exact LP)
x_lp = cp.Variable(n)
constraints_lp = [x_lp >= 0, x_lp <= 1] + [x_lp[u] + x_lp[v] >= 1 for (u, v) in E]
prob_lp = cp.Problem(cp.Minimize(cp.sum(x_lp)), constraints_lp)

prob_lp.solve(solver=cp.GLPK)

x_star = np.array(x_lp.value).flatten()
lp_opt = prob_lp.value

S_half = np.where(x_star >= 0.5 - tol)[0]
cost_half = len(S_half)

print("\n=== (b) LP + 1/2-rounding ===")
print("LP optimum value:", lp_opt)
print("Rounded cover size:", cost_half)
print("Rounded is cover?", is_vertex_cover(S_half, E))
print("Ratio rounded / ILP:", cost_half / cost_ilp if cost_ilp > 0 else None)


# (c) Incremental rule (should be subset of S_half)
S_inc = set()
for (u, v) in E:
    if (u in S_inc) or (v in S_inc):
        continue
    if x_star[u] >= x_star[v]:
        S_inc.add(u)
    else:
        S_inc.add(v)

S_inc = np.array(sorted(S_inc), dtype=int)
cost_inc = len(S_inc)

print("\n=== (c) Incremental rule ===")
print("Incremental cover size:", cost_inc)
print("Incremental is cover?", is_vertex_cover(S_inc, E))
print("Incremental subset of S_half?", set(S_inc).issubset(set(S_half)))
print("Compare to 1/2-rounding (should be <=):", cost_inc, "<=", cost_half)
print("Ratio incremental / ILP:", cost_inc / cost_ilp if cost_ilp > 0 else None)


Loaded graph with n=100, |E|=490
Installed solvers: ['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

=== (a) ILP ===
ILP optimum (size): 70
ILP is cover? True

=== (b) LP + 1/2-rounding ===
LP optimum value: 50.0
Rounded cover size: 100
Rounded is cover? True
Ratio rounded / ILP: 1.4285714285714286

=== (c) Incremental rule ===
Incremental cover size: 93
Incremental is cover? True
Incremental subset of S_half? True
Compare to 1/2-rounding (should be <=): 93 <= 100
Ratio incremental / ILP: 1.3285714285714285


## Question 4 — Vertex Cover (ILP vs LP Relaxation + Rounding)

We are given a graph \(G=(V,E)\) from the data file. In this instance the loaded graph has
\[
|V|=100,\qquad |E|=490.
\]



### (a) ILP formulation and optimal solution

Let \(x_i \in \{0,1\}\) for each vertex \(i\in V\), where \(x_i=1\) means that vertex \(i\) is included in the vertex cover.

The ILP for minimum vertex cover is:
\[
\min \sum_{i\in V} x_i
\]
subject to, for every edge \((u,v)\in E\),
\[
x_u + x_v \ge 1,
\]
and
\[
x_i\in\{0,1\}\quad \forall i\in V.
\]

**Result (CVXPY + GLPK\_MI):**
- Optimal ILP value (minimum vertex cover size): **70**
- The returned set is a valid vertex cover.



### (b) LP relaxation and 1/2-rounding

The LP relaxation replaces integrality by bounds:
\[
0 \le x_i \le 1\quad \forall i\in V.
\]

So the LP is:
\[
\min \sum_{i\in V} x_i
\quad \text{s.t.}\quad
x_u+x_v\ge 1\ \forall (u,v)\in E,\quad
0\le x_i\le 1.
\]

Let \(x^\*\) be the optimal LP solution. The standard rounding rule discussed in class is:
\[
S_{1/2}=\{\, i\in V : x_i^\* \ge 1/2 \,\}.
\]

This produces a vertex cover because for every edge \((u,v)\),
\(x_u^\*+x_v^\*\ge 1\) implies at least one endpoint has value \(\ge 1/2\).

**Result (CVXPY + GLPK):**
- LP optimum value: **50.0**
- 1/2-rounded cover size: **100**
- This rounded set is a valid vertex cover.
- Comparison to ILP optimum:  
  \[
  \frac{|S_{1/2}|}{|S_{\text{ILP}}|}=\frac{100}{70}\approx 1.4286.
  \]

So the LP gives a lower bound (50.0) on the ILP optimum (70), but the simple 1/2-rounding is quite conservative here and ends up selecting all vertices.


### (c) Incremental rounding rule, correctness, and comparison

Incremental rule: start with \(S=\emptyset\). Scan edges in any order.  
If an edge \((u,v)\) is already covered by \(S\), do nothing.  
Otherwise add \(u\) if \(x_u^\* \ge x_v^\*\), else add \(v\).

**Why it is a vertex cover:**  
Whenever an uncovered edge \((u,v)\) is encountered, the algorithm adds one endpoint, so that edge becomes covered immediately. Since every edge is processed, all edges end up covered.

**Why it has cost no more than the 1/2-rounding solution:**  
If \((u,v)\) is uncovered, we know \(x_u^\* + x_v^\* \ge 1\), hence
\(\max(x_u^\*,x_v^\*) \ge 1/2\).  
The rule adds the endpoint with larger \(x^\*\), which must therefore have value at least \(1/2\).  
So every vertex the incremental rule adds is also included in \(S_{1/2}\), meaning
\[
S_{\text{inc}} \subseteq S_{1/2}
\quad \Rightarrow \quad
|S_{\text{inc}}| \le |S_{1/2}|.
\]

**Result (using the computed LP solution \(x^\*\)):**
- Incremental cover size: **93**
- This set is a valid vertex cover.
- It is a subset of the 1/2-rounded cover and indeed satisfies \(93 \le 100\).
- Comparison to ILP optimum:
  \[
  \frac{|S_{\text{inc}}|}{|S_{\text{ILP}}|}=\frac{93}{70}\approx 1.3286.
  \]



### Final comparison

- ILP optimum (best possible): **70**
- LP optimum value (lower bound on ILP): **50.0**
- 1/2-rounding cover size: **100**  (ratio \(\approx 1.4286\))
- Incremental cover size: **93**     (ratio \(\approx 1.3286\))

In this graph instance, the incremental rounding rule improves over the basic 1/2-rounding, but it is still above the optimal ILP solution.
